In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from typing import List, Tuple
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from pathlib import Path
from random import choices
import warnings
warnings.filterwarnings("ignore") # Ignore all warnings

# Gershgorin circle theorem

In [ ]:
def add_gaussian_noise(matrix:np.array, mean:float = 0, std_dev:float = 10**-3) -> np.array:
    """
    Adds Gaussian noise to the off-diagonal elements of a given matrix.

    Parameters:
        matrix (np.array): The matrix to which noise will be added.
        mean (float, optional): The mean of the Gaussian noise. Default is 0.
        std_dev (float, optional): The standard deviation of the Gaussian noise. Default is 0.001.

    Returns:
        np.array: The noisy matrix with updated off-diagonal elements.

    This function iterates over the off-diagonal elements of the input matrix and adds Gaussian noise to each element.
    """
    noisy_matrix = np.copy(matrix)
    rows, cols = matrix.shape
    for i in range(rows):
        for j in range(cols):
            if i != j:  # Only update off-diagonal elements
                noisy_matrix[i, j] += np.random.normal(mean, std_dev)
    return noisy_matrix

def get_Kronecker_Factors(N:int, layer_index:int, max_steps:int, choices_list:bool = None, 
                          noise:bool = False, std:float = 10**-2) -> Tuple:
    """
    Retrieves Kronecker factors for specified layers and optionally adds Gaussian noise.

    Parameters:
        N (int): Number of layer to retrieve for.
        layer_index (int): Index of the layer to retrieve factors for.
        max_steps (int): Maximum step interval to sample from.
        choices_list (bool, optional): If provided, uses this list to sample steps. Otherwise, generates a new list.
        noise (bool, optional): If True, adds Gaussian noise to the matrices. Default is False.
        std (float, optional): Standard deviation of the Gaussian noise. Default is 0.01.

    Returns:
        tuple: A tuple containing lists of H_bar and S matrices, the list of chosen steps, and the class name of the layer.

    Raises:
        FileNotFoundError: If the directories for H-bar or S are not found.

    This function can apply Gaussian noise to both H_bar and S matrices if specified, affecting only off-diagonal elements.
    """
    H_bar_dir = Path("H_bar_resnet").expanduser()
    S_dir = Path("S_resnet").expanduser()
    if not H_bar_dir.exists() or not S_dir.exists:
        raise FileNotFoundError(f'H_bar_resnet and S_resnet directories not found')
    if choices_list is None:
        choices_list = choices(range(50, max_steps + 50, 50), k=N)
    H_bar, S = [], []
    for i in choices_list:
        with open(f'H_bar_resnet/H_bar_{i}.pkl', 'rb') as f:
            dict_H_bar = pickle.load(f)
        H_bar_l = list(dict_H_bar.values())[layer_index]
        if noise:
            H_bar.append(add_gaussian_noise(H_bar_l.cpu().numpy(), std_dev=std))
        else:
            H_bar.append(H_bar_l.cpu().numpy())
        with open(f'S_resnet/S_{i}.pkl', 'rb') as f:
            dict_S = pickle.load(f)
        S_l = list(dict_S.values())[layer_index]
        layer_name = list(dict_S.keys())[layer_index].__class__.__name__
        if noise:
            S.append(add_gaussian_noise(S_l.cpu().numpy(), std_dev=std))
        else:
            S.append(S_l.cpu().numpy())
    return H_bar, S, choices_list, layer_name

def plot_gershgorin_disks(H_bar:List, S:List, choices_list:List, N:int, 
                          layer_name:str, layer_index:int, verbose:bool = False):
    """
    Plots Gershgorin disks for H-bar and S matrices of a specified layer to analyze their eigenvalues.

    Parameters:
        H_bar (List): List of H_bar matrices.
        S (List): List of S matrices.
        choices_list (List): Steps at which the matrices were sampled.
        N (int): Number of layers to plot.
        layer_name (str): Name of the layer.
        layer_index (int): Index of the layer in the model.
        verbose (bool, optional): If True, adds a detailed title to the plots.

    This function visualizes the eigenvalue distribution and the Gershgorin disks, which estimate the eigenvalue bounds, for each matrix.
    Eigenvalues are plotted as red 'x' marks, and Gershgorin disks are shown as circles on the complex plane.
    """
    fig, axs = plt.subplots(2, N, figsize=(2*N, 2*N))
    colormap = plt.cm.viridis

    # Calculate normalization factors separately for H and S matrices
    norm_A = Normalize(vmin=min([np.sum(np.abs(m), axis=1).min() for m in H_bar]), vmax=max([np.sum(np.abs(m), axis=1).max() for m in H_bar]))
    norm_G = Normalize(vmin=min([np.sum(np.abs(m), axis=1).min() for m in S]), vmax=max([np.sum(np.abs(m), axis=1).max() for m in S]))

    # ScalarMappables for creating the color bars
    sm_A = ScalarMappable(norm=norm_A, cmap=colormap)
    sm_G = ScalarMappable(norm=norm_G, cmap=colormap)

    def plot_single_matrix(matrix, ax, index, type, norm):
        eigenvalues = np.linalg.eigvals(matrix)
        radius_values = [np.sum(np.abs(matrix[i, :])) - np.abs(matrix[i, i]) for i in range(len(matrix))]
        for i, radius in enumerate(radius_values):
            center = matrix[i, i]
            color = colormap(norm(radius))
            circle = Circle((center, 0), radius, color=color, fill=False, alpha=0.5)
            ax.add_artist(circle)
            ax.plot(center, 0, 'ko')
        ax.plot(eigenvalues.real, eigenvalues.imag, 'rx')
        if type == "$\\mathcal{{S}}$":
            ax.set_xlabel('Real Part', fontsize=10)
        if index == 0:
            ax.set_ylabel('Imaginary Part', fontsize=10)
        ax.set_title(f'Matrix {type} | Step {choices_list[index]}', fontsize=10, fontweight='bold')
        ax.set_aspect('equal')
        ax.grid(True)
        ax.set_xlim([np.min(matrix.diagonal() - np.sum(np.abs(matrix), axis=1)), np.max(matrix.diagonal() + np.sum(np.abs(matrix), axis=1))])
        ax.set_ylim([-np.max(np.sum(np.abs(matrix), axis=1)), np.max(np.sum(np.abs(matrix), axis=1))])
        asp = np.diff(ax.get_xlim())[0] / np.diff(ax.get_ylim())[0]
        ax.set_aspect(asp)

    # Plot Gershgorin disks for H and S matrices
    for i, matrix in enumerate(H_bar):
        plot_single_matrix(matrix, axs[0, i], i, "$\\mathcal{{H}}$", norm_A)
    for i, matrix in enumerate(S):
        plot_single_matrix(matrix, axs[1, i], i, "$\\mathcal{{S}}$", norm_G)

    if verbose:
        plt.suptitle(f'Gershgorin Disks for Matrices $\\mathcal{{H}}$ and $\\mathcal{{S}}$ of {layer_name} layer at position {layer_index+1} of Resnet18',fontsize=10, fontweight='bold')

    # Add color bars for H and S
    cbar_ax_A = fig.add_axes([0.91, 0.56, 0.02, 0.30])
    cbar_ax_G = fig.add_axes([0.91, 0.12, 0.02, 0.30])
    fig.colorbar(sm_A,cax=cbar_ax_A)
    fig.colorbar(sm_G,cax=cbar_ax_G)
    plt.subplots_adjust(wspace=0.4, hspace=0.3)
    plt.show()

In [ ]:
# ---- Settings from Paper ----
N = 2  # Number of matrices for H_bar and for S
layer_index = 40 # 36 or 40
max_steps = 9800
choice_list = [5200, 9800]
# ---- Settings from Paper ----
H_bar, S, choices_list, layer_name = get_Kronecker_Factors(N, layer_index, max_steps, choice_list, noise=False)
plot_gershgorin_disks(H_bar, S, choices_list, N, layer_name, layer_index)

# Perturbation Analysis

In [ ]:
def plot_eigenvalues(H_bar:List, H_bar_noise:List, S:List, S_noise:List, choices_list:List, 
                     N:int, layer_name:str, layer_index:int, verbose:bool = False):
    """
    Plots the eigenvalues of matrices with and without added noise, alongside a reference line based on the Kaiser Rule.

    Parameters:
        H_bar (List): List of original H-bar matrices.
        H_bar_noise (List): List of H-bar matrices with noise added.
        S (List): List of original S matrices.
        S_noise (List): List of S matrices with noise added.
        choices_list (List): Steps at which the matrices were sampled.
        N (int): Number of layers to plot.
        layer_name (str): Name of the layer.
        layer_index (int): Index of the layer in the model.
        verbose (bool, optional): If True, adds a detailed title to the plots.

    This function visualizes the eigenvalues of the matrices on logarithmic scales for both axes, allowing for detailed 
    comparison between original and noisy conditions. It highlights the stability or instability introduced by noise.
    """
    fig, axs = plt.subplots(2, N, figsize=(2*N, 2*N))
    def plot_single_matrix(matrix,matrix_noise, ax, index, type):
        # Calculate the eigenvalues
        eigenvalues = np.linalg.eigh(matrix)[0]  # Use only the eigenvalues
        eigenvalues_noise = np.linalg.eigh(matrix_noise)[0]
        ax.plot(eigenvalues, '-', color='blue', markersize=6, linewidth=2, label="Without Noise")  
        ax.plot(eigenvalues_noise, '--', color='red', markersize=6, linewidth=2, label="With Noise")  
        ax.axhline(y=1, color='green', linestyle='--', linewidth=1.5, label="Kaiser Rule")  
        ax.set_title(f'Matrix {type} | Step {choices_list[index]}', fontsize=10, fontweight='bold')
        ax.set_yscale('log')
        ax.set_xscale('log')
        if type == "$\\mathcal{{S}}$":
            ax.set_xlabel('$\\lambda$', fontsize=10)
        ax.grid(True, which='both', linestyle='--', linewidth=0.5) 
 
    for i, (matrix, matrix_noise) in enumerate(zip(H_bar, H_bar_noise)):
        plot_single_matrix(matrix,matrix_noise, axs[0, i], i, "$\\mathcal{{H}}$")
    for i, (matrix, matrix_noise) in enumerate(zip(S, S_noise)):
        plot_single_matrix(matrix, matrix_noise, axs[1, i], i, "$\\mathcal{{S}}$")
    if verbose:
        plt.suptitle(f'Comparison of Eigenvalues: Original vs. Noisy $\\mathcal{{H}}$ and $\\mathcal{{S}}$ Matrices of {layer_name} layer at position {layer_index+1} of Resnet18', fontsize=16, fontweight='bold')
    plt.subplots_adjust(wspace=0.4, hspace=0.4)
    plt.show()

In [ ]:
# ---- Settings from Paper ----
N = 2  # Number of matrices for H_bar and for S
layer_index = 40 # 36 or 40
max_steps = 9800
choice_list = [5200, 9800]
# ---- Settings from Paper ----
A, G, choices_list, layer_name = get_Kronecker_Factors(N, layer_index, max_steps, choice_list)
A_noise, G_noise, choices_list, layer_name = get_Kronecker_Factors(N, layer_index, max_steps, choice_list, noise=True, std=10**-3)
plot_eigenvalues(A,A_noise, G, G_noise, choices_list, N, layer_name, layer_index)

# FFT and SNR analysis

In [ ]:
def calculate_snr(signal:np.array, noise:np.array) -> np.array:
    """
    Calculates the Signal-to-Noise Ratio (SNR) in decibels (dB) for given signal and noise arrays.

    Parameters:
        signal (np.array): The main signal array.
        noise (np.array): The noise array mixed with the signal.

    Returns:
        np.array: The SNR of the signal in decibels.

    This function computes the SNR by first calculating the power of the signal and noise, then using these 
    to compute the logarithmic ratio of signal power to noise power.
    """
    power_signal = np.mean(signal ** 2)
    power_noise = np.mean(noise ** 2)
    return 10 * np.log10(power_signal / power_noise)

def plot_fft(H_bar:List, H_bar_noise:List, S:List, S_noise:List, choices_list:List, N:int, 
             layer_name:str, layer_index:int, verbose:bool = False):
    """
    Plots the Fast Fourier Transform (FFT) of matrices and their noisy counterparts, and calculates the SNR.

    Parameters:
        H_bar (List): List of original H-bar matrices.
        H_bar_noise (List): List of H-bar matrices with added noise.
        S (List): List of original S matrices.
        S_noise (List): List of S matrices with added noise.
        choices_list (List): List of steps at which the matrices were sampled.
        N (int): Number of layers to plot.
        layer_name (str): Name of the layer.
        layer_index (int): Index of the layer in the model.
        verbose (bool, optional): If True, adds a detailed title to the plots.

    This function visualizes the magnitude spectrum of the FFT for each matrix, providing insights into 
    the frequency components of both original and noisy matrices. It also annotates each plot with the calculated SNR values.
    """
    fig, axs = plt.subplots(4, N, figsize=(4*N, 6*N)) 

    def plot_single_matrix(matrix, ax, index, type):
        fft_matrix = np.fft.fftshift(np.fft.fft2(matrix))
        magnitude_spectrum = 20*np.log(np.abs(fft_matrix))
        img = ax.imshow(magnitude_spectrum, cmap='hot', extent=[-np.pi, np.pi, -np.pi, np.pi])
        ax.set_title(f'Matrix {type} | Step {choices_list[index]}', fontweight='bold')
        asp = np.diff(ax.get_xlim())[0] / np.diff(ax.get_ylim())[0]
        ax.set_aspect(asp)
        return img

    # Plot H_bar matrices and calculate SNR
    snrs_A = []
    for i, (matrix, matrix_noise) in enumerate(zip(H_bar, H_bar_noise)):
        imgs_A = plot_single_matrix(matrix, axs[0, i], i, "$\\mathcal{{H}}$")
        _ = plot_single_matrix(matrix_noise, axs[1, i], i, "$\\hat{{\\mathcal{{H}}}}$")
        snr_value = calculate_snr(np.diag(matrix), np.triu(matrix_noise,1))
        snrs_A.append(snr_value)
        axs[1, i].annotate(f'SNR: {snr_value:.2f} dB', xy=(0.5, -0.25), xycoords='axes fraction', ha='center', va='center', fontweight='bold', fontsize=16)

    # Plot S matrices and calculate SNR
    snrs_G = []
    for i, (matrix, matrix_noise) in enumerate(zip(S, S_noise)):
        imgs_G = plot_single_matrix(matrix, axs[2, i], i, "$\\mathcal{{S}}$")
        _ = plot_single_matrix(matrix_noise, axs[3, i], i, "$\\hat{{\\mathcal{{S}}}}$")
        snr_value = calculate_snr(np.diag(matrix), np.triu(matrix_noise,1))
        snrs_G.append(snr_value)
        axs[3, i].annotate(f'SNR: {snr_value:.2f} dB', xy=(0.6, -0.25), xycoords='axes fraction', ha='center', va='center', fontweight='bold', fontsize=16)

    # Create a color bar for the H_bar matrices
    cax_A = fig.add_axes([0.78, 0.55, 0.02, 0.30])  
    fig.colorbar(imgs_A, cax=cax_A)
    # Create a color bar for the S matrices
    cax_G = fig.add_axes([0.78, 0.14, 0.02, 0.30]) 
    fig.colorbar(imgs_G, cax=cax_G)

    if verbose:
        plt.suptitle(f'Comparison of FFT: Original vs. Noisy $\\mathcal{{H}}$ and $\\mathcal{{S}}$ Matrices of {layer_name} layer at position {layer_index+1}', fontsize=16, fontweight='bold')
    plt.subplots_adjust(wspace=-0.4, hspace=0.5)
    plt.show()

In [ ]:
# ---- Settings from Paper ----
N = 2  # Number of matrices for H_bar and for S
layer_index = 40 # 36 or 40
max_steps = 9800
choice_list = [5200, 9800]
# ---- Settings from Paper ----
A, G, choices_list, layer_name = get_Kronecker_Factors(N, layer_index, max_steps, choice_list)
A_noise, G_noise, choices_list, layer_name = get_Kronecker_Factors(N, layer_index, max_steps, choice_list, noise=True, std=10**-3)
plot_fft(A,A_noise, G, G_noise, choices_list, N, layer_name, layer_index)